In [1]:
import numpy as np
import navpy
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from Project_Framework import Satellite

def makesphere(radius, resolution=10):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v)
    Y = radius * np.sin(u)*np.sin(v)
    Z = radius * np.cos(v)
    return (X, Y, Z)

def Constellation(a,e,i,aop,num_sats,num_planes,num_periods=1,dt=10):
    constellation = []
    d_raan = 360/num_planes
    planes = {round(i,4):0 for i in np.arange(0,360,d_raan)}
    O = 0
    idx = num_sats
    while idx > 0:
        idx -= 1
        planes[round(O,4)]+=1
        O += d_raan
        O = O%360

    for O in planes:
        fs = np.linspace(0,360,planes[O]+1)
        for f in fs[:-1]:
            sat = Satellite()
            sat.from_kepler_elements(a,e,i,O,aop,f)
            sat.Propagate(0,num_periods*sat.T,dt)
            constellation.append(sat)
    return constellation

def Plot_3d_Constellations(constellations):
    i,j,k = makesphere(6378,resolution=100)

    fig = go.Figure(data=[go.Surface(x=i,y=j,z=k,
                                    colorscale=[[0, "blue"], [1,"blue"]],
                                    opacity=0.25,showscale=False)])
    
    rp = max([max([sat.a*(1-sat.e) for sat in constellation]) for constellation in constellations])
    
    colors = px.colors.qualitative.Plotly

    for constellation_num,constellation in enumerate(constellations,start=1):
        color = colors[constellation_num%len(colors)]
        for sat_num,sat in enumerate(constellation,start=1):
            if sat_num == 1:
                fig.add_trace(go.Scatter3d(x=sat.TLE[:,0],y=sat.TLE[:,1],z=sat.TLE[:,2],line=dict(width=2,color=color),mode="lines",name=f"Constellation {constellation_num}"))
            else:
                fig.add_trace(go.Scatter3d(x=sat.TLE[:,0],y=sat.TLE[:,1],z=sat.TLE[:,2],line=dict(width=2,color=color),mode="lines",name=f"Constellation {constellation_num}",showlegend=False))
            fig.add_trace(go.Scatter3d(x=[sat.TLE[0,0]],y=[sat.TLE[0,1]],z=[sat.TLE[0,2]],marker=dict(size=5,color=color),showlegend=False,name=f"Constellation {sat_num}"))

    fig['layout']['scene']['aspectmode'] = "cube"
    fig['layout']['scene']['xaxis']['range'] = [-rp,rp]
    fig['layout']['scene']['yaxis']['range'] = [-rp,rp]
    fig['layout']['scene']['zaxis']['range'] = [-rp,rp]
    fig.show()
    return fig

def Plot_3d(sats):
    i,j,k = makesphere(6378,resolution=100)

    fig = go.Figure(data=[go.Surface(x=i,y=j,z=k,
                                    colorscale=[[0, "blue"], [1,"blue"]],
                                    opacity=0.25,showscale=False)])
    
    rp = max([sat.a*(1-sat.e) for sat in sats])
    
    colors = px.colors.qualitative.Plotly

    for sat_num,sat in enumerate(sats,start=1):
        color = colors[sat_num%len(colors)]
        fig.add_trace(go.Scatter3d(x=sat.TLE[:,0],y=sat.TLE[:,1],z=sat.TLE[:,2],line=dict(width=2,color=color),mode="lines",name=f"Satellite {sat_num}"))
        fig.add_trace(go.Scatter3d(x=[sat.TLE[0,0]],y=[sat.TLE[0,1]],z=[sat.TLE[0,2]],marker=dict(size=5,color=color),showlegend=False,name=f"Satellite {sat_num}"))

    fig['layout']['scene']['aspectmode'] = "cube"
    fig['layout']['scene']['xaxis']['range'] = [-rp,rp]
    fig['layout']['scene']['yaxis']['range'] = [-rp,rp]
    fig['layout']['scene']['zaxis']['range'] = [-rp,rp]
    fig.show()
    return fig

def Ground_Track_Constellation(constellations):
    fig = go.Figure()
    colors = px.colors.qualitative.Plotly
    for constellation_num,constellation in enumerate(constellations,start=1):
        color = colors[constellation_num%len(colors)]
        for sat_num,sat in enumerate(constellation,start=1):
            rECEF,vECEF = sat.ECEF_TLE()
            lat,lon,alt = navpy.ecef2lla(1000*rECEF)
            lat = np.nan_to_num(lat,0)
            if sat_num == 1:
                fig.add_trace(go.Scattergeo(lon=lon,lat=lat,marker=dict(size=1,color=color),name=f"Constellation {constellation_num}",line=dict(width=2),mode="lines"))
            else:
                fig.add_trace(go.Scattergeo(lon=lon,lat=lat,marker=dict(size=1,color=color),name=f"Constellation {constellation_num}",line=dict(width=2),mode="lines",showlegend=False))
    fig.show()
    return fig

def Ground_Track(sats):
    fig = go.Figure()
    for sat_num,sat in enumerate(sats,start=1):
        rECEF,vECEF = sat.ECEF_TLE()
        lat,lon,alt = navpy.ecef2lla(1000*rECEF)
        lat = np.nan_to_num(lat,0)
        fig.add_trace(go.Scattergeo(lon=lon,lat=lat,marker=dict(size=1),name=f"Satellite {sat_num}",line=dict(width=2),mode="lines"))
    fig.show()
    return fig

def Rotational_Plots(sats):
    fig = make_subplots(rows=3, cols=2)

    for sat_num,sat in enumerate(sats,start=1):
        for idx in range(1,4):
            fig.add_trace(go.Scatter(x=sat.times,
                                    y=np.degrees(sat.TLE[:,idx+5])%360,
                                    name=f"Satellite {sat_num}"),
                                    row=idx,col=1)
            
        for idx in range(1,4):
            fig.add_trace(go.Scatter(x=sat.times,
                                    y=np.degrees(sat.TLE[:,idx+8]),
                                    name=f"Satellite {sat_num}"),
                                    row=idx,col=2)

    fig.show()
    return fig

In [ ]:
constellations = []
alt = 2000
constellations.append(Constellation(6378+alt,0,60,0,1,1,num_periods=1,dt=1))

# fig1 = Plot_3d_Constellations(constellations)
fig2 = Ground_Track_Constellation(constellations)

In [ ]:
import pandas as pd
import plotly.express as px

df = {'x':[],'y':[],'z':[],'const_list':[],'sat_list':[]}
for constellation_num,constellation in enumerate(constellations):
    for sat_num,sat in enumerate(constellation):
        df['x'].extend(sat.TLE[:,0])
        df['y'].extend(sat.TLE[:,1])
        df['z'].extend(sat.TLE[:,2])
        df['const_list'].extend(len(sat.TLE)*[constellation_num])
        df['sat_list'].extend(len(sat.TLE)*[sat_num])
df = pd.DataFrame(df)

px.scatter3d(df,x="x",y="y",z="z",color="const_list")